## Import libraries and get username/password

In [1]:
import mysql.connector
from mysql.connector import errorcode
import pandas as pd
import difflib
from difflib import SequenceMatcher as SM
from IPython.display import clear_output

#Reads in your input for user name and password
myuser = input("User Name: ")
mypassw = input("Password: ")

User Name: root
Password: twigthegreat


## Connect to the database

In [2]:
#This tries to connect to the database
try:
    cnx = mysql.connector.connect(user = myuser, password = mypassw,
                              host = '127.0.0.1',
                              database = 'datacleandb')
    print("connected")
except mysql.connector.Error as err:
  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Something is wrong with your user name or password")
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("Database does not exist")
  else:
    print(err)

connected


## Create lookup and mistake tables

In [3]:
#Create the lookup Table
cursor = cnx.cursor()
sqlcommand = """CREATE TABLE IF NOT EXISTS mydata_lookup(
                Valid_Sub_Vendor VARCHAR(255) PRIMARY KEY,
                is_Supplier_Name BOOL)"""
cursor.execute(sqlcommand, multi = False)
#Creates the mistake table, so that we can compare to common mistakes
sqlcommand = """CREATE TABLE IF NOT EXISTS mydata_lookup_mistakes(
                Correct_Vendor VARCHAR(255),
                Mistake_Vendor VARCHAR(255),
                FOREIGN KEY(Correct_Vendor) REFERENCES mydata_lookup(Valid_Sub_Vendor),
                PRIMARY KEY(Correct_Vendor, Mistake_Vendor))"""
cursor.execute(sqlcommand, multi = False)
cnx.commit()

## TEMP: add values to Lookup and Mistake tables

In [ ]:
#---DO NOT RUN MULTIPLE TIMES ---
#TEMP ADDING TO THE LOOKUP TABLE
sqlcommand = """INSERT INTO mydata_lookup 
                VALUES ('Stars Prime', FALSE)"""
cursor.execute(sqlcommand, multi = False)
cnx.commit()

In [ ]:
#---DO NOT RUN MULTIPLE TIMES---
#TEMP ADDING TO THE MISTAKE TABLE
sqlcommand = """INSERT INTO mydata_lookup_mistakes 
                VALUES ('Claridge', 'Claridges')"""
cursor.execute(sqlcommand, multi = False)
cnx.commit()

# Start of Program Code

## import lookup and mistake from MySQL

In [4]:
#Function to import the lookup table
def importLookupTable(connection):
    sqlcommand = 'SELECT * FROM mydata_lookup'
    lookup_table = pd.DataFrame(columns = ['Valid_Sub_Vendor','is_Supplier_Name'])
    try:
        lookup_table = pd.read_sql(sqlcommand, con = connection)
    except:
        print("mydata_lookup table is empty")
    return lookup_table

#Function to import the mistake table
def importMistakeTable(connection):
    sqlcommand = 'SELECT * FROM mydata_lookup_mistakes'
    mistake_table = pd.DataFrame(columns = ['Correct_Vendor','Mistake_Vendor'])
    try:
        mistake_table = pd.read_sql(sqlcommand, con = connection)
    except:
        print("mydata_lookup_mistake table is empty")
    return mistake_table

## Finds % similarity in lookup

In [5]:
#Name matching in lookup table
def nameMatchingLookup(val, lookup_df):
    #Creates the empty dataframe of ratios
    ratio_df = pd.DataFrame(columns = ['Ratio','LookupVal','MistakeVal'])
    #Makes the value to compare lowercase
    val = val.lower()

    #Iterates through all of the rows in the lookup table
    for index, row in lookup_df.iterrows():
        #Runs the sequence matcher
        sub = SM( a = val, b = row['Valid_Sub_Vendor'].lower() )
        #Saves the ratio and name into the dataframe
        ratio_df.loc[index] = [round(sub.ratio(), 3), row['Valid_Sub_Vendor'], None]

    #Returns the ratio dataframe sorted
    return ratio_df.sort_values(by = ['Ratio'], ascending = False)

## Finds % similarity in mistake

In [6]:
#Name matching in mistake table
def nameMatchingMistake(val, mistake_df):
    #Creates the empty dataframe of ratios
    ratio_df = pd.DataFrame(columns = ['Ratio','LookupVal','MistakeVal'])
    #Makes the value to compare lowercase
    val = val.lower()

    #Iterates through all of the rows in the mistake table
    for index, row in mistake_df.iterrows():
        #Runs the sequence matcher
        sub = SM( a = val, b = row['Mistake_Vendor'].lower() )
        #Saves the ratio and name into the dataframe
        ratio_df.loc[index] = [round(sub.ratio(), 3), row['Correct_Vendor'], row['Mistake_Vendor']]

    #Returns the ratio dataframe sorted
    return ratio_df.sort_values(by = ['Ratio'], ascending = False)

## Merges lookup and mistake %'s

In [7]:
#Gets all the ratio's, combines and sorts
def nameMatching(rowVal, lookup_df, mistake_df):
    #Compare the value to the ones in the tables
    ratio_l_df = nameMatchingLookup(rowVal, lookup_df)
    ratio_m_df = nameMatchingMistake(rowVal, mistake_df)
    
    #Merge the two tables together, remove 'duplicates' keep max
    ratio_df = pd.concat([ratio_l_df, ratio_m_df], ignore_index = True)
    ratio_df = ratio_df.loc[ratio_df.reset_index().groupby(['LookupVal'])['Ratio'].idxmax()]
    ratio_df = ratio_df.sort_values(by = ['Ratio'], ascending = False)
    return ratio_df.reset_index(drop = True)

##### Checks to see if a value is a number

In [8]:
#Checks if it is a number
def isNumber(s):
    try:
        int(s)
        return True
    except ValueError:
        return False 

##### User Interface GUI for fakeGUI

In [9]:
def theUI(name1, name2, ind):
    if ind == 1:
        print("======================================================")
        print(name1)
        print("------------------------------------------------------")
        print("c          : Create Custom Value")
        print("<num>      : Choose row # <num>")
        print("<other>    : Anything else to see next set of rows")
        print("------------------------------------------------------")
        print("")
    elif ind == 2:
        print("======================================================")
        print(name1)
        print(name2)
        print("------------------------------------------------------")
        print("c          : Use the Custom Value")
        print("<num>      : Choose row # <num> instead of custom")
        print("<other>    : Anything else to see next set of rows")
        print("------------------------------------------------------")
        print("")


## Makes the User Interface (also used by customValue())

In [10]:
#Fake GUI - NOTE: customValue() also uses this. BEWARE OF CHANGES 
def fakeGUI(ratio, name, indicator, name2):
    topindex = ratio['Ratio'].count() - 1
    show = 5 #The number of lookup's to show at a time
    x = 0
    
    #print("++++c: custom, <num>: the row, else: see next row++++")
    #print("Current: ", name)
    while True:
        clear_output()
        theUI(name, name2, indicator)
        #Makes sure to bound our index
        top = x + show - 1
        #print("X: ", x, ", ", "TOP: ", top, ", IND: ", topindex)#0, 1, 2
        if top > topindex:
            top = topindex
            
        #Print the current rows in the result
        l = [i for i in range(x,top+1)]
        print(ratio.iloc[l])

        #Loops until we get a valid user entry
        while True:
            #Get the user input
            #val = input("Which (if any) is correct: ")
            val = input("Choice: ")
            
            #Check to see if the input is valid numerically
            if isNumber(val):
                #Value is valid, between 0 and the last one
                #if int(val) >= x and int(val) <= top:
                if int(val) >= 0 and int(val) <= topindex:
                    break
            elif val == 'c':
                val = -2 #Temp value to indicate we want a custom value
                break
            else:
                val = -1 #Temp value to indicate we continue with next ones
                break
        #END LOOP
        
        #Returns the value if it is legit
        #With a value of -1, we know that we just continue
        #with the next ratio values
        if val != -1:
            return val
        
        #Increment the x index
        x = top + 1
        if x > topindex:
            #Houston, we have a problem
            x = 0
    #END LOOP

## Adds mistake to local and MySQL

In [11]:
#Insert the new mistake into the mistake table
def addMistake(mistake, actual, df):
    #mistake: mistake value, actual: value in lookup table, df: mistake_df table
    
    #Make sure the mistake is not the same value as the actual
    if mistake == actual:
        return df
    
    #Check if the value already exists in the table
    for i, r in df.iterrows():
        if (r.iloc[0] == actual) & (r.iloc[1] == mistake):
            return df

    #Add the mistake to the server
    try:
        sqlcommand = "INSERT INTO mydata_lookup_mistakes VALUES ( (SELECT Valid_Sub_Vendor FROM mydata_lookup WHERE Valid_Sub_Vendor = \"" + actual + "\"), \"" + mistake + "\")"
        cursor.execute(sqlcommand, multi = False)
        cnx.commit()
    except:
        print("Error adding Mistake value to MySQL: ", mistake, " Actual: ", actual) #------------------------------------------------------------------------------------------------
    

    #Adds the mistake locally then returns it
    return df.append({'Correct_Vendor' : actual, 'Mistake_Vendor' : mistake}, ignore_index = True)

## Adds lookup value to local and MySQL

In [12]:
#Insert the new value into the lookup table
def addLookup(val, df, isSupplier):

    #Check if the value already exists in the table
    for i, r in df.iterrows():
        if r.iloc[0] == val:
            return df

    #Add the lookup value to the server
    try:
        truth = "FALSE"
        if isSupplier:
            truth = "TRUE"
        #sqlcommand = "INSERT INTO mydata_lookup VALUES ('" + val + "', " + truth + ")"
        sqlcommand = "INSERT INTO mydata_lookup VALUES (\"" + val + "\", " + truth + ")"
        cursor.execute(sqlcommand, multi = False)
        cnx.commit()
    except:
        print("Error adding Lookup value to MySQL: ", val)

    sup = 0
    if isSupplier:
        sup = 1
    
    #Adds the lookup locally then returns it
    return df.append({'Valid_Sub_Vendor' : val, 'is_Supplier_Name' : sup}, ignore_index = True)

## Run when a user wants to use a custom value

In [13]:
def customValue(l_df, m_df, name):
    keep = 'y'
    cust = ''
    #0: using custom, 1: using prev & update mistake table, 2: using prev
    vect = [0, "lookup", "mistake"] #["type", "lookup", "mistake"]
    while True:
        #Gets the custom value from the user
        cust = input("Custom: ")
        keep = input("Keep Value (y/n): ")
        if keep == 'y':
            break
    
    #CHECK TO SEE IF THE USER WANTS A ALREADY MADE ONE----------
    ratio_df = nameMatching(cust, l_df, m_df)
    
    #print("-----------------------------------------------------")
    #GUI: Do something with the information you have received
    result = int(fakeGUI(ratio_df, name, 2, cust))
    
    if result == -2:
        #Use the custom typed one
        vect = [0, cust, name]
    else:
        #Use the one already there
        if ratio_df.iloc[result,0] != 1:
            #not a previously made mistake: insert into local and mysql mistake table
            vect = [1, ratio_df.iloc[result,1], name]
        else:
            vect = [2, ratio_df.iloc[result,1], None]
    #END CHECK-------------------------------------------------
    
    return vect

### Queries the database and populates the lookup table from old data

In [14]:
def poplookup():
    try:
        #UPDATES the lookup table with previously made values from mydata table
        sqlcommand = """INSERT INTO mydata_lookup( Valid_Sub_Vendor, is_Supplier_Name )
        SELECT DISTINCT Audit_Dashboard_Vendor, FALSE
        FROM mydata
        WHERE Audit_Dashboard_Vendor is not NULL AND
        Audit_Dashboard_Vendor != Supplier_name"""
        cursor.execute(sqlcommand, multi = False)
        cnx.commit()
    except:
        print("Danger Will Robinson, Danger!")

### Queries the database and populates the mistake table from old data

In [15]:
def popmistake():
    try:
        #UPDATES the lookup table with previously made values from mydata table
        sqlcommand = """INSERT INTO mydata_lookup_mistakes( Mistake_Vendor, Correct_Vendor )
        SELECT DISTINCT Sub_Vendor, Audit_Dashboard_Vendor
        FROM mydata
        WHERE Audit_Dashboard_Vendor is not NULL AND
        Audit_Dashboard_Vendor != Supplier_name AND
        Sub_Vendor is not NULL"""
        cursor.execute(sqlcommand, multi = False)
        cnx.commit()
    except:
        print("Danger Will Robinson, Danger2!")

### Adds the value to mydata database

In [27]:
def addDatabase(origVal, goodVal):
    #Insert the change into mydata.Audit_Dashboard_Vendor
    try:
        sqlcommand = """UPDATE mydata
        SET Audit_Dashboard_Vendor = \"%s\"
        WHERE Sub_Vendor = \"%s\"""" %(goodVal, origVal)
        cursor.execute(sqlcommand, multi = False)
        cnx.commit()
    except:
        print("Adding to mydata has a problem")

# Main Program Loop

### Populate the lookup/mistake tables from old data

In [ ]:
poplookup()
popmistake()

### Stuff before comparing

In [31]:
#Import the lookup and mistake tables
lookup_df = importLookupTable(cnx)
mistake_df = importMistakeTable(cnx)

print("TODO: make the NULL Sub_Vendor rows auto complete")

#EX: SELECT * FROM tbl LIMIT 5,10;  # Retrieve rows 6-15

#Gets how many rows there are in the database
sqlcommand = "SELECT COUNT(Sub_Vendor) FROM mydata"
s = pd.read_sql(sqlcommand, con = cnx)
dbSize = s.iloc[0]['COUNT(Sub_Vendor)']

mysql_index = 0 #Starting index
chunksize = 2 #How many we get at a time from the database

TODO: make the NULL Sub_Vendor rows auto complete


### Iterating through the database rows

In [34]:
#loop through the rows in MySQL by 'chunksize' at a time
while True:
    #Get the non-null and non-completed rows # Audit_Dashboard_Vendor IS NULL AND
    sqlcommand = "SELECT Sub_Vendor, Audit_Dashboard_Vendor as ADV FROM mydata WHERE Sub_Vendor IS NOT NULL LIMIT"
    #Adds which rows are to be used onto the SQL command
    sqlcommand = sqlcommand + " " + str(mysql_index) + "," + str(chunksize)
    mydata_df = pd.read_sql(sqlcommand, con = cnx)
    print("======================================================")

    #Loop through the rows to clean (non NULL)
    for index, row in mydata_df.iterrows():
        if row['ADV'] != None:
            continue
        #Generates all the ratios [rows of: ['Ratio','LookupVal','MistakeVal']]
        ratio_df = nameMatching(row['Sub_Vendor'], lookup_df, mistake_df)
        
        #Auto chooses for the user if there is one that perfectly matches
        result = 0
        if ratio_df.empty or ratio_df.iloc[result, 0] < 1:
            #GUI: Do something with the information you have received
            result = int(fakeGUI(ratio_df, row['Sub_Vendor'],1,""))
        else:
            print("Perfect match found: ", ratio_df.iloc[result, 1], " for ", row['Sub_Vendor'])

        if result >= 0: #User chose a row
            if ratio_df.iloc[result,0] != 1:
                #not a previously made mistake: insert into local and mysql mistake table
                mistake_df = addMistake(row['Sub_Vendor'], ratio_df.iloc[result,1], mistake_df)
            #Insert change into the database
            addDatabase(row[0], ratio_df.iloc[result, 1])
        elif result == -2: #User chose to make custom result
            #choice format ["type", "lookupVal", "mistakeVal"]
            choice = customValue(lookup_df, mistake_df, row['Sub_Vendor'])
            #0: using custom, 1: using prev & update mistake table, 2: using prev
            if choice[0] == 0: #Use the custom typed one
                #Add it to the lookup table
                lookup_df = addLookup(choice[1], lookup_df, False)
                #Add it to the mistake table
                mistake_df = addMistake(row['Sub_Vendor'], choice[1], mistake_df)
            elif choice[0] == 1: #Use one already in lookup
                #Add it to the lookup table
                lookup_df = addLookup(choice[1], lookup_df, False)
                #Add it to the mistake table
                mistake_df = addMistake(row['Sub_Vendor'], choice[1], mistake_df)
            else: #has to be 2  #Use one already in lookup
                #Add it to the lookup table
                lookup_df = addLookup(choice[1], lookup_df, False)
            #Insert change into the database
            addDatabase(row[0], choice[1])
        #END ELIF
    #END LOOP
    
    #Allow the user to quit now that the chunk of data is completed
    print("======================================================")
    quit = input("Quit? (y/n):")
    if quit == "y":
        break
    #clear_output()
    #Done with current data from database
    mysql_index = mysql_index + chunksize
    if mysql_index >= dbSize: #no more items in the database
        break
    if (mysql_index + chunksize) >= dbSize: #can't take the full chunksize
        chunksize = dbSize - mysql_index
    
#END LOOP
#END OF THE ACTUAL PROGRAM-----------------------------------------------------------

Decipher Tools
Decipher Tools
------------------------------------------------------
c          : Use the Custom Value
<num>      : Choose row # <num> instead of custom
<other>    : Anything else to see next set of rows
------------------------------------------------------

   Ratio        LookupVal    MistakeVal
0  0.640            Hertz   Hertz Tolls
1  0.538           Budget  Budget Tolls
2  0.500             Avis    Avis Tolls
3  0.500   Kimpton Hotels          None
4  0.483  DELTA AIR LINES          None
Choice: c
Quit? (y/n):y


#### Close connections when you are done

In [63]:
#Closes the connection to the database and the cursor
cursor.close()
cnx.close()

### Random code I was testing

In [26]:
#Insert the change into mydata.Audit_Dashboard_Vendor
goodVal = "Hillstone"
origVal = "Hillstone"
try:
    sqlcommand = """UPDATE mydata
    SET Audit_Dashboard_Vendor = \"%s\"
    WHERE Sub_Vendor = \"%s\"""" %(goodVal, origVal)
    cursor.execute(sqlcommand, multi = False)
    cnx.commit()
except:
    print("Adding to mydata has a problem")

print(sqlcommand)

UPDATE mydata
    SET Audit_Dashboard_Vendor = "Hillstone"
    WHERE Sub_Vendor = "Hillstone"
